This notebook evaluates the results on first 20 rows, with explanations available, of the MEDMCQA validation dataset (https://huggingface.co/datasets/medmcqa/viewer/default/validation) as follows

* Model: GPT-3.5-Turbo-0613 (using https://chat.lmsys.org/ for generating answers)
* CoT strategies: Zero and few shot prompts
* Evaluation: sentence BERT comparison, GPT-4 comparison, KG based, ROUGE/BLEU

### Common imports

In [80]:
import itertools
import torch

In [2]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("all-MiniLM-L6-v2")

/home/ajayago/anaconda3/envs/huggingface/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/ajayago/anaconda3/envs/huggingface/lib/python3.8/site-packages/torch/cuda/__init__.py:141: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11010). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at /opt/conda/conda-bld/pytorch_1708025845899/work/c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


In [3]:
import evaluate
bleu_score = evaluate.load("bleu")
rouge_score = evaluate.load('rouge')

In [4]:
import pandas as pd
semmed_db = pd.read_csv("/data/ajayago/KB_embeddings/dataset/polypharmacy/PREDICATIONS_OCCURS.csv")
semmed_db.head()

,PREDICATE,SUBJECT_CUI,SUBJECT_NAME,SUBJECT_SEMTYPE,SUBJECT_NOVELTY,SUBJECT_SCORE,OBJECT_CUI,OBJECT_NAME,OBJECT_SEMTYPE,OBJECT_NOVELTY,OBJECT_SCORE,OCCURS
0,ADMINISTERED_TO,C0000248,2-Acetylaminofluorene,hops,1.0,1000,C0018561,Mesocricetus auratus,mamm,1.0,1000,2
1,ADMINISTERED_TO,C0000248,2-Acetylaminofluorene,hops,1.0,1000,C0034693,Rattus norvegicus,mamm,1.0,861,31
2,ADMINISTERED_TO,C0000248,2-Acetylaminofluorene,hops,1.0,1000,C0034693,Rattus norvegicus,mamm,1.0,1000,31
3,ADMINISTERED_TO,C0000275,2-Chloroadenosine,phsu,1.0,1000,C0034693,Rattus norvegicus,mamm,1.0,1000,2
4,ADMINISTERED_TO,C0000407,3-Hydroxyanthranilic Acid,bacs,1.0,1000,C0029974,Ovum,cell,1.0,888,1


In [5]:
semmed_db["SUBJECT_NAME"] = semmed_db["SUBJECT_NAME"].apply(lambda x: x.upper())
semmed_db["OBJECT_NAME"] = semmed_db["OBJECT_NAME"].apply(lambda x: x.upper())

In [6]:
semmed_db.head()

,PREDICATE,SUBJECT_CUI,SUBJECT_NAME,SUBJECT_SEMTYPE,SUBJECT_NOVELTY,SUBJECT_SCORE,OBJECT_CUI,OBJECT_NAME,OBJECT_SEMTYPE,OBJECT_NOVELTY,OBJECT_SCORE,OCCURS
0,ADMINISTERED_TO,C0000248,2-ACETYLAMINOFLUORENE,hops,1.0,1000,C0018561,MESOCRICETUS AURATUS,mamm,1.0,1000,2
1,ADMINISTERED_TO,C0000248,2-ACETYLAMINOFLUORENE,hops,1.0,1000,C0034693,RATTUS NORVEGICUS,mamm,1.0,861,31
2,ADMINISTERED_TO,C0000248,2-ACETYLAMINOFLUORENE,hops,1.0,1000,C0034693,RATTUS NORVEGICUS,mamm,1.0,1000,31
3,ADMINISTERED_TO,C0000275,2-CHLOROADENOSINE,phsu,1.0,1000,C0034693,RATTUS NORVEGICUS,mamm,1.0,1000,2
4,ADMINISTERED_TO,C0000407,3-HYDROXYANTHRANILIC ACID,bacs,1.0,1000,C0029974,OVUM,cell,1.0,888,1


In [7]:
import networkx as nx
# load Semmed DB into memory
G = nx.from_pandas_edgelist(semmed_db, source="SUBJECT_NAME", target="OBJECT_NAME", edge_attr="OCCURS", edge_key="PREDICATE")

In [8]:
print(nx.number_of_nodes(G), nx.number_of_edges(G))

296597 14087555


In [9]:
concepts = set(nx.nodes(G))
len(concepts)

296597

In [15]:
from datasets import load_dataset
medmcqa_data = load_dataset("medmcqa", split="validation")

### Question 1

```
Question: Which of the following is not true about glomerular capillaries?
Option A: The oncotic pressure of the fluid leaving the capillaries is less than that of fluid entering it
Option B: Glucose concentration in the capillaries is the same as that in glomerular filtrate
Option C: Constriction of afferent aeriole decreases the blood flow to the glomerulas
Option D: Hematocrit of the fluid leaving the capillaries is less than that of the fluid entering it
Answer: A
Exp: Ans-a. The oncotic pressure of the fluid leaving the capillaries is less than that of fluid entering it Guyton I LpJ1 4-.;anong 23/e p653-6_)Glomerular oncotic pressure (due to plasma protein content) is higher than that of filtrate oncotic pressure in Bowman's capsule"Since glucose is freely filtered and the fluid in the Bowman's capsule is isotonic with plasma, the concentration of glucose in the filtrate is the same as in the capillaries
```

In [91]:
print(f"Question: {medmcqa_data['question'][1]}")
print(f"Option A: {medmcqa_data['opa'][1]}")
print(f"Option B: {medmcqa_data['opb'][1]}")
print(f"Option C: {medmcqa_data['opc'][1]}")
print(f"Option D: {medmcqa_data['opd'][1]}")
print(f"Answer: {['A', 'B', 'C', 'D'][medmcqa_data['cop'][1]]}")

Question: Which of the following is not true about glomerular capillaries')
Option A: The oncotic pressure of the fluid leaving the capillaries is less than that of fluid entering it
Option B: Glucose concentration in the capillaries is the same as that in glomerular filtrate
Option C: Constriction of afferent aeriole decreases the blood flow to the glomerulas
Option D: Hematocrit of the fluid leaving the capillaries is less than that of the fluid entering it
Answer: A


#### Zero Shot Prompt 

Prompt:
```
Question: Which of the following is not true about glomerular capillaries?
Option A: The oncotic pressure of the fluid leaving the capillaries is less than that of fluid entering it
Option B: Glucose concentration in the capillaries is the same as that in glomerular filtrate
Option C: Constriction of afferent aeriole decreases the blood flow to the glomerulas
Option D: Hematocrit of the fluid leaving the capillaries is less than that of the fluid entering it
Answer: 
```

Answer:
```
The answer is option B: Glucose concentration in the capillaries is the same as that in glomerular filtrate.
```


**Sentence BERT comparison**

In [92]:
generated_cot = "The answer is option B: Glucose concentration in the capillaries is the same as that in glomerular filtrate."
cot_gt = """Ans-a. The oncotic pressure of the fluid leaving the capillaries is less than that of fluid entering it Guyton I LpJ1 4-.;anong 23/e p653-6_)Glomerular oncotic pressure (due to plasma protein content) is higher than that of filtrate oncotic pressure in Bowman's capsule"Since glucose is freely filtered and the fluid in the Bowman's capsule is isotonic with plasma, the concentration of glucose in the filtrate is the same as in the capillaries"""
sentences_to_compare = [
    generated_cot,
    cot_gt
]
sentence_embeddings = model.encode(sentences_to_compare)


In [93]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_similarity(sentence_embeddings[[0]], sentence_embeddings[[1]])

array([[0.7343774]], dtype=float32)

**GPT-4 comparison** (based on https://chat.lmsys.org/ `gpt-4-1106-preview`)

Prompt:
```
How similar are the following sentences? Rate the similarity using a score between 0 and 1 with a reasoning for the same.
Sentence A: The answer is option B: Glucose concentration in the capillaries is the same as that in glomerular filtrate.
Sentence B: Ans-a. The oncotic pressure of the fluid leaving the capillaries is less than that of fluid entering it Guyton I LpJ1 4-.;anong 23/e p653-6_)Glomerular oncotic pressure (due to plasma protein content) is higher than that of filtrate oncotic pressure in Bowman's capsule"Since glucose is freely filtered and the fluid in the Bowman's capsule is isotonic with plasma, the concentration of glucose in the filtrate is the same as in the capillaries
```
Response:
```
The similarity between Sentence A and Sentence B is relatively low, and I would rate it around 0.3.

Both sentences are discussing physiological aspects related to the capillaries and glomerular filtrate, which suggests a thematic similarity. They mention properties of substances in the blood and glomerular filtrate, but they focus on different concepts:

- Sentence A states that the glucose concentration is the same in the capillaries and the glomerular filtrate.
- Sentence B discusses oncotic pressure differences and mentions that the glomerular oncotic pressure is higher than that of the filtrate in Bowman's capsule. It also notes that glucose is freely filtered and that the filtrate is isotonic with plasma, which implies a similarity in glucose concentrations between the capillaries and Bowman's capsule filtrate, but this is not the main focus of the sentence.

The reasoning for the similarity score is as follows:

- Both sentences talk about the properties of substances within the capillaries and filtrate, which provides a basic thematic similarity.
- Sentence A explicitly states the similarity in glucose concentration between two points, while Sentence B implies this information in a more complex explanation about oncotic pressure.
- The main topic of Sentence A is glucose concentration, while Sentence B focuses on oncotic pressure, although it includes a statement that indirectly supports the idea in Sentence A.

Thus, while there is some overlap in the content, the primary focuses of the sentences are different, and Sentence B includes additional information and context not present in Sentence A. This justifies the score being closer to 0 than to 1.
```

**Knowledge Graph based comparison**

This needs a few key phrases in the text to be highlighted by user - this can be offloaded to a biomedical NER system later. These phrases must also be present in the knowledge graph used.

Currently, key phrases are being extracted using GPT-4 from chat.lmsys.org or are manually created. The manual ones have a higher chance of being found in SemmedDB.

After identifying the key terms in the Q and all options, shortest paths are found for these concepts in the KG. For the terms found, the overlap with each option is found. The option with most overlap in terms is the chosen answer from the KG.

In [94]:
optionA_phrases = [w.upper() for w in medmcqa_data["opa"][1].split(" ")]
optionB_phrases = [w.upper() for w in medmcqa_data["opb"][1].split(" ")]
optionC_phrases = [w.upper() for w in medmcqa_data["opc"][1].split(" ")]
optionD_phrases = [w.upper() for w in medmcqa_data["opd"][1].split(" ")]

In [95]:
# phrases from generated cot that could be connected in the KG
candidate_key_phrases = ["glucose", "glucose concentration", "capillary", "capillaries", "glomerular", "filterate", "oncotic pressure", "aeriole", "hematocrit"] # manually generated
# candidate_key_phrases = list(set(optionA_phrases) | set(optionB_phrases) | set(optionC_phrases) | set(optionD_phrases))
print(candidate_key_phrases)
# candidate_key_phrases = ["glomerular capillaries", # from GPT-4
# "oncotic pressure",
# "glucose concentration",
# "glomerular filtrate",
# "constriction of afferent arteriole",
# "blood flow to the glomerulus",
# "hematocrit"]
key_phrases = []
for candidate in candidate_key_phrases:
    if candidate.upper() in concepts:
        key_phrases.append(candidate.upper())
print(key_phrases)


['glucose', 'glucose concentration', 'capillary', 'capillaries', 'glomerular', 'filterate', 'oncotic pressure', 'aeriole', 'hematocrit']
['GLUCOSE', 'ONCOTIC PRESSURE']


In [96]:
# only if more than 2 concepts are present in the KG
if len(key_phrases) < 2:
    print("Cannot perform this analysis without atleast 2 concepts present in the KG")
else:
    # find shortest paths that exist between the concepts
    combinations_of_concepts = itertools.combinations(key_phrases, 2)
    paths = []
    for combination in combinations_of_concepts:
        shortest_paths = list(nx.all_shortest_paths(G, source=combination[0], target=combination[1]))
        paths.extend(shortest_paths)
    if len(paths) == 0:
        print("No paths exist between the concepts")
    else:
        print("Paths exist between the concepts")
        print(paths)

Paths exist between the concepts
[['GLUCOSE', 'CONTRACTION', 'ONCOTIC PRESSURE'], ['GLUCOSE', 'LIPOGENESIS', 'ONCOTIC PRESSURE'], ['GLUCOSE', 'EXCRETORY FUNCTION', 'ONCOTIC PRESSURE'], ['GLUCOSE', 'NEOPLASM', 'ONCOTIC PRESSURE'], ['GLUCOSE', 'KIDNEY FAILURE, ACUTE', 'ONCOTIC PRESSURE'], ['GLUCOSE', 'LYMPH FLOW', 'ONCOTIC PRESSURE'], ['GLUCOSE', 'ALBUMINS', 'ONCOTIC PRESSURE'], ['GLUCOSE', 'PHARMACEUTICAL PREPARATIONS', 'ONCOTIC PRESSURE'], ['GLUCOSE', 'WATER', 'ONCOTIC PRESSURE'], ['GLUCOSE', 'CHLORIDES', 'ONCOTIC PRESSURE'], ['GLUCOSE', 'FATTY ACIDS, NONESTERIFIED', 'ONCOTIC PRESSURE'], ['GLUCOSE', 'SODIUM CHLORIDE', 'ONCOTIC PRESSURE'], ['GLUCOSE', 'POTASSIUM', 'ONCOTIC PRESSURE'], ['GLUCOSE', 'STARCH', 'ONCOTIC PRESSURE'], ['GLUCOSE', 'PROTEINS', 'ONCOTIC PRESSURE'], ['GLUCOSE', 'ENDOTOXINS', 'ONCOTIC PRESSURE'], ['GLUCOSE', 'ELECTROLYTES', 'ONCOTIC PRESSURE'], ['GLUCOSE', 'ANGIOTENSINS', 'ONCOTIC PRESSURE'], ['GLUCOSE', 'IONS', 'ONCOTIC PRESSURE'], ['GLUCOSE', 'THYROXINE', 'ONCOTIC

In [97]:
counts_dict_per_option = {
    "A": 0, "B": 0, "C": 0, "D": 0
}
optionA = " ".join([w.upper() for w in medmcqa_data["opa"][1].split(" ")])
optionB = " ".join([w.upper() for w in medmcqa_data["opb"][1].split(" ")])
optionC = " ".join([w.upper() for w in medmcqa_data["opc"][1].split(" ")])
optionD = " ".join([w.upper() for w in medmcqa_data["opd"][1].split(" ")])
for path in paths:
    for term in path:
        if term in optionA:
            counts_dict_per_option["A"] += 1
        if term in optionB:
            counts_dict_per_option["B"] += 1
        if term in optionC:
            counts_dict_per_option["C"] += 1
        if term in optionD:
            counts_dict_per_option["D"] += 1
print(counts_dict_per_option)

{'A': 82, 'B': 82, 'C': 0, 'D': 0}


In [98]:
max(counts_dict_per_option, key=counts_dict_per_option.get) # answer from KG - B also has same number of terms as A

'A'

**Graph Edit distance**

In [99]:
# number of terms in each node of G
max_words = 1
for w in concepts:
    if len(w.split(" ")) > max_words:
        max_words = len(w.split(" "))
        print(w)
print(max_words)

RESPIRATORY ORGAN
AVIAN ENCEPHALOMYELITIS VIRUS ANTIBODY
CLEAVAGE AND POLYADENYLATION SPECIFICITY FACTOR, 30-KDA SUBUNIT
APICAL RAMUS OF TRUNK OF APICAL SEGMENTAL ARTERY
MECHANICAL FRAGMENTATION OF CATARACT WITH EXTRACTION BY POSTERIOR ROUTE
OXIDOREDUCTASE ACTIVITY, ACTING ON SINGLE DONORS WITH INCORPORATION OF MOLECULAR OXYGEN
DIPTHERIA + TETANUS + PERTUSSIS + POLIOMYELITIS + HAEMOPHILUS INFLUENZAE B VACCINE
INSULIN, ISOPHANE, HUMAN 70 UNT/ML / INSULIN, REGULAR, HUMAN 30 UNT/ML INJECTABLE SUSPENSION [HUMAN MIXTARD PENFILL]
OXIDOREDUCTASE ACTIVITY, ACTING ON SINGLE DONORS WITH INCORPORATION OF MOLECULAR OXYGEN, INCORPORATION OF TWO ATOMS OF OXYGEN
REPAIR OF DOUBLE OUTLET RIGHT VENTRICLE WITH ANASTOMOSIS OF LEFT VENTRICLE TO AORTA VIA VENTRICULAR SEPTAL DEFECT AND DIRECT ANASTOMOSIS OF RIGHT VENTRICLE TO PULMONARY TRUNK
26


In [100]:
# Evaluating CoT
# candidate_cot_terms = [w.upper() for w in generated_cot.split(" ")]
candidate_cot_terms = ["glucose", "concentration", "capillaries", "glomerular", "filtrate"]
cot_terms = []
for candidate in candidate_cot_terms:
    if candidate.upper() in concepts:
        cot_terms.append(candidate.upper())
print(cot_terms)

['GLUCOSE']


In [101]:
# create graph chaining these terms in the order in which they occur.
cot_graph = nx.DiGraph()
cot_graph.add_nodes_from(cot_terms)
for c in cot_terms:
    if cot_terms.index(c) < len(cot_terms) - 1:
        cot_graph.add_edge(c, cot_terms[cot_terms.index(c) + 1])
print(list(cot_graph.nodes()))
print(list(cot_graph.edges()))

['GLUCOSE']
[]


In [102]:
# graph edit distance
gedistances = []
for path in paths:
    path_graph = nx.DiGraph()
    path_graph.add_nodes_from(path)
    for c in path:
        if path.index(c) < len(path) - 1:
            path_graph.add_edge(c, path[path.index(c) + 1])
    gedistances.append(nx.graph_edit_distance(path_graph, cot_graph))
print(gedistances)

[4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0]


In [103]:
print(max(gedistances))

4.0


**Comparison of CoT embedding and paths from KG**

Combine the triples from KG paths and compare the cosine similarity of resulting ClinicalBERT embedding against generated CoT embedding.

In [104]:
from transformers import AutoTokenizer, AutoModel
clinicalBert_tokenizer = AutoTokenizer.from_pretrained("medicalai/ClinicalBERT")
clinicalBert_model = AutoModel.from_pretrained("medicalai/ClinicalBERT")

In [105]:
clinicalbert_path_embeddings = []
for path in paths:
    # print(path)
    inputs = clinicalBert_tokenizer(" ".join(path), return_tensors="pt")
    outputs = clinicalBert_model(**inputs)
    clinicalbert_path_embeddings.append(outputs.last_hidden_state.mean(dim=1))
print(torch.cat(clinicalbert_path_embeddings, dim=0).shape)
print(len(paths))


torch.Size([82, 768])
82


In [106]:
generated_cot_clinicalbert_embedding = clinicalBert_model(**clinicalBert_tokenizer(generated_cot, return_tensors="pt")).last_hidden_state.mean(dim=1)
generated_cot_clinicalbert_embedding.shape

torch.Size([1, 768])

In [107]:
similarity_cot_graph_paths = []
for i in clinicalbert_path_embeddings:
    cosine_sim = (torch.nn.functional.cosine_similarity(generated_cot_clinicalbert_embedding, i))
    similarity_cot_graph_paths.append(cosine_sim.item())
print(max(similarity_cot_graph_paths))

0.7254173755645752


**BLEU/Rouge**

In [108]:
bleu = bleu_score.compute(predictions=[generated_cot], references=[cot_gt])
rouge = rouge_score.compute(predictions=[generated_cot], references=[cot_gt])
print(bleu)
print(rouge)

{'bleu': 0.007737618711910635, 'precisions': [0.7, 0.3157894736842105, 0.2222222222222222, 0.058823529411764705], 'brevity_penalty': 0.033373269960326066, 'length_ratio': 0.22727272727272727, 'translation_length': 20, 'reference_length': 88}
{'rouge1': 0.30927835051546393, 'rouge2': 0.12631578947368421, 'rougeL': 0.2268041237113402, 'rougeLsum': 0.2268041237113402}


#### Zero Shot Prompt with CoT prompt
Prompt:
```
Question: Which of the following is not true about glomerular capillaries?
Option A: The oncotic pressure of the fluid leaving the capillaries is less than that of fluid entering it
Option B: Glucose concentration in the capillaries is the same as that in glomerular filtrate
Option C: Constriction of afferent aeriole decreases the blood flow to the glomerulas
Option D: Hematocrit of the fluid leaving the capillaries is less than that of the fluid entering it
Answer: Let's think step by step
```

Answer:
```
Let's analyze each option:

Option A: The oncotic pressure of the fluid leaving the capillaries is less than that of fluid entering it.
This statement is true. The oncotic pressure in the glomerular capillaries is higher than in the Bowman's capsule, which causes fluid to be filtered out of the capillaries.

Option B: Glucose concentration in the capillaries is the same as that in glomerular filtrate.
This statement is not true. Glucose is actively reabsorbed in the proximal tubule, so the concentration of glucose in the capillaries is lower than in the glomerular filtrate.

Option C: Constriction of afferent arteriole decreases the blood flow to the glomerulus.
This statement is true. Constriction of the afferent arteriole reduces the blood flow entering the glomerulus, decreasing the glomerular filtration rate.

Option D: Hematocrit of the fluid leaving the capillaries is less than that of the fluid entering it.
This statement is true. The glomerular capillaries filter out smaller molecules and fluid, while leaving behind larger molecules and formed elements, such as red blood cells. This results in a lower hematocrit in the fluid leaving the capillaries compared to the entering fluid.

Therefore, the answer is Option B: Glucose concentration in the capillaries is the same as that in glomerular filtrate.
```

**Sentence BERT comparison**

In [109]:
generated_cot = """
Let's analyze each option:

Option A: The oncotic pressure of the fluid leaving the capillaries is less than that of fluid entering it.
This statement is true. The oncotic pressure in the glomerular capillaries is higher than in the Bowman's capsule, which causes fluid to be filtered out of the capillaries.

Option B: Glucose concentration in the capillaries is the same as that in glomerular filtrate.
This statement is not true. Glucose is actively reabsorbed in the proximal tubule, so the concentration of glucose in the capillaries is lower than in the glomerular filtrate.

Option C: Constriction of afferent arteriole decreases the blood flow to the glomerulus.
This statement is true. Constriction of the afferent arteriole reduces the blood flow entering the glomerulus, decreasing the glomerular filtration rate.

Option D: Hematocrit of the fluid leaving the capillaries is less than that of the fluid entering it.
This statement is true. The glomerular capillaries filter out smaller molecules and fluid, while leaving behind larger molecules and formed elements, such as red blood cells. This results in a lower hematocrit in the fluid leaving the capillaries compared to the entering fluid.

Therefore, the answer is Option B: Glucose concentration in the capillaries is the same as that in glomerular filtrate.
"""
cot_gt = """Ans-a. The oncotic pressure of the fluid leaving the capillaries is less than that of fluid entering it Guyton I LpJ1 4-.;anong 23/e p653-6_)Glomerular oncotic pressure (due to plasma protein content) is higher than that of filtrate oncotic pressure in Bowman's capsule"Since glucose is freely filtered and the fluid in the Bowman's capsule is isotonic with plasma, the concentration of glucose in the filtrate is the same as in the capillaries"""
sentences_to_compare = [
    generated_cot,
    cot_gt
]
sentence_embeddings = model.encode(sentences_to_compare)


In [110]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_similarity(sentence_embeddings[[0]], sentence_embeddings[[1]])

array([[0.80747664]], dtype=float32)

**GPT-4 comparison**
Prompt:
```
How similar are the following sentences? Rate the similarity using a score between 0 and 1 with a reasoning for the same.
Sentence A: Let's analyze each option:Option A: The oncotic pressure of the fluid leaving the capillaries is less than that of fluid entering it.This statement is true. The oncotic pressure in the glomerular capillaries is higher than in the Bowman's capsule, which causes fluid to be filtered out of the capillaries.Option B: Glucose concentration in the capillaries is the same as that in glomerular filtrate.This statement is not true. Glucose is actively reabsorbed in the proximal tubule, so the concentration of glucose in the capillaries is lower than in the glomerular filtrate.Option C: Constriction of afferent arteriole decreases the blood flow to the glomerulus.This statement is true. Constriction of the afferent arteriole reduces the blood flow entering the glomerulus, decreasing the glomerular filtration rate.Option D: Hematocrit of the fluid leaving the capillaries is less than that of the fluid entering it.This statement is true. The glomerular capillaries filter out smaller molecules and fluid, while leaving behind larger molecules and formed elements, such as red blood cells. This results in a lower hematocrit in the fluid leaving the capillaries compared to the entering fluid.Therefore, the answer is Option B: Glucose concentration in the capillaries is the same as that in glomerular filtrate.
Sentence B: Ans-a. The oncotic pressure of the fluid leaving the capillaries is less than that of fluid entering it Guyton I LpJ1 4-.;anong 23/e p653-6_)Glomerular oncotic pressure (due to plasma protein content) is higher than that of filtrate oncotic pressure in Bowman's capsule"Since glucose is freely filtered and the fluid in the Bowman's capsule is isotonic with plasma, the concentration of glucose in the filtrate is the same as in the capillaries
```
Response
```
The similarity between Sentence A and Sentence B can be considered moderate to high, and I would rate it around 0.7 to 0.8.

Both sentences are discussing physiological concepts related to the capillaries and glomerular filtrate, and there is significant overlap in the content. Here's the reasoning for the similarity score:

1. Both sentences address the concept of oncotic pressure in the capillaries and its comparison with the fluid in Bowman's capsule, agreeing that the oncotic pressure is higher in the glomerular capillaries.

2. Both sentences reference the concentration of glucose in the capillaries and the glomerular filtrate. Sentence A states that the glucose concentration in the capillaries is lower than in the glomerular filtrate due to reabsorption in the proximal tubule, while Sentence B asserts that the concentration is the same because glucose is freely filtered and the filtrate is isotonic. This is a contradiction, but it appears that Sentence A contains an error in the conclusion, as the supporting explanation acknowledges the isotonic nature of the filtrate with plasma, which implies equality in glucose concentration.

3. Sentence A provides additional information and an error in its conclusion, which is not present in Sentence B. Sentence A also includes information about the effects of afferent arteriole constriction and hematocrit changes, which are not mentioned in Sentence B.

The main reason the score is not closer to 1 is due to the contradiction in Sentence A's conclusion about glucose concentration, which seems to be an error as it contradicts the explanation provided in the same sentence. If we consider the explanatory parts of Sentence A that align with Sentence B, the similarity is quite high. However, the additional information and the erroneous conclusion in Sentence A affect the overall similarity score.
```

**Graph Edit distance**

In [111]:
# Evaluating CoT
# candidate_cot_terms = [w.upper() for w in generated_cot.split(" ")]
candidate_cot_terms = ["glucose", "concentration", "capillaries", "glomerular", "filtrate", "oncotic pressure", "afferent arteriole", "blood flow", "glomerulus", "hematocrit", "fluid", "Bowman's capsule", "proximal tubule"]
cot_terms = []
for candidate in candidate_cot_terms:
    if candidate.upper() in concepts:
        cot_terms.append(candidate.upper())
print(cot_terms)

['GLUCOSE', 'ONCOTIC PRESSURE', 'BLOOD FLOW']


In [112]:
# create graph chaining these terms in the order in which they occur.
cot_graph = nx.DiGraph()
cot_graph.add_nodes_from(cot_terms)
for c in cot_terms:
    if cot_terms.index(c) < len(cot_terms) - 1:
        cot_graph.add_edge(c, cot_terms[cot_terms.index(c) + 1])
print(list(cot_graph.nodes()))
print(list(cot_graph.edges()))

['GLUCOSE', 'ONCOTIC PRESSURE', 'BLOOD FLOW']
[('GLUCOSE', 'ONCOTIC PRESSURE'), ('ONCOTIC PRESSURE', 'BLOOD FLOW')]


In [113]:
# graph edit distance
gedistances = []
for path in paths:
    path_graph = nx.DiGraph()
    path_graph.add_nodes_from(path)
    for c in path:
        if path.index(c) < len(path) - 1:
            path_graph.add_edge(c, path[path.index(c) + 1])
    gedistances.append(nx.graph_edit_distance(path_graph, cot_graph))
print(gedistances)

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [114]:
print(max(gedistances))

0.0


**Comparison of CoT embedding and paths from KG**

Combine the triples from KG paths and compare the cosine similarity of resulting ClinicalBERT embedding against generated CoT embedding.

In [115]:
from transformers import AutoTokenizer, AutoModel
clinicalBert_tokenizer = AutoTokenizer.from_pretrained("medicalai/ClinicalBERT")
clinicalBert_model = AutoModel.from_pretrained("medicalai/ClinicalBERT")

In [116]:
clinicalbert_path_embeddings = []
for path in paths:
    # print(path)
    inputs = clinicalBert_tokenizer(" ".join(path), return_tensors="pt")
    outputs = clinicalBert_model(**inputs)
    clinicalbert_path_embeddings.append(outputs.last_hidden_state.mean(dim=1))
print(torch.cat(clinicalbert_path_embeddings, dim=0).shape)
print(len(paths))


torch.Size([82, 768])
82


In [117]:
generated_cot_clinicalbert_embedding = clinicalBert_model(**clinicalBert_tokenizer(generated_cot, return_tensors="pt")).last_hidden_state.mean(dim=1)
generated_cot_clinicalbert_embedding.shape

torch.Size([1, 768])

In [118]:
similarity_cot_graph_paths = []
for i in clinicalbert_path_embeddings:
    cosine_sim = (torch.nn.functional.cosine_similarity(generated_cot_clinicalbert_embedding, i))
    similarity_cot_graph_paths.append(cosine_sim.item())
print(max(similarity_cot_graph_paths))

0.615809977054596


**BLEU/Rouge**

In [119]:
bleu = bleu_score.compute(predictions=[generated_cot], references=[cot_gt])
rouge = rouge_score.compute(predictions=[generated_cot], references=[cot_gt])
print(bleu)
print(rouge)

{'bleu': 0.14082294100265094, 'precisions': [0.23076923076923078, 0.1630901287553648, 0.1206896551724138, 0.08658008658008658], 'brevity_penalty': 1.0, 'length_ratio': 2.659090909090909, 'translation_length': 234, 'reference_length': 88}
{'rouge1': 0.3875432525951557, 'rouge2': 0.27177700348432055, 'rougeL': 0.31141868512110726, 'rougeLsum': 0.31833910034602075}


#### Zero shot CoT with template
Prompt:
```
1. <step1>
2. <step2>
...
So the answer is (<answer>).
Make sure that the answer uses the above format and answers the question step by step.
Question: Which of the following is not true about glomerular capillaries?
Option A: The oncotic pressure of the fluid leaving the capillaries is less than that of fluid entering it
Option B: Glucose concentration in the capillaries is the same as that in glomerular filtrate
Option C: Constriction of afferent aeriole decreases the blood flow to the glomerulas
Option D: Hematocrit of the fluid leaving the capillaries is less than that of the fluid entering it
Answer: 

```

Answer:
```
Step 1: Evaluate Option A: The oncotic pressure of the fluid leaving the capillaries is less than that of fluid entering it.
This statement is true. The oncotic pressure in the glomerular capillaries is higher than in the Bowman's capsule, which causes fluid to be filtered out of the capillaries.

Step 2: Evaluate Option B: Glucose concentration in the capillaries is the same as that in glomerular filtrate.
This statement is not true. Glucose is actively reabsorbed in the proximal tubule, so the concentration of glucose in the capillaries is lower than in the glomerular filtrate.

Step 3: Evaluate Option C: Constriction of afferent arteriole decreases the blood flow to the glomerulus.
This statement is true. Constriction of the afferent arteriole reduces the blood flow entering the glomerulus, decreasing the glomerular filtration rate.

Step 4: Evaluate Option D: Hematocrit of the fluid leaving the capillaries is less than that of the fluid entering it.
This statement is true. The glomerular capillaries filter out smaller molecules and fluid, while leaving behind larger molecules and formed elements, such as red blood cells. This results in a lower hematocrit in the fluid leaving the capillaries compared to the entering fluid.

Therefore, the answer is Option B: Glucose concentration in the capillaries is the same as that in glomerular filtrate.
```

**Sentence BERT comparison**

In [120]:
generated_cot = """
Step 1: Evaluate Option A: The oncotic pressure of the fluid leaving the capillaries is less than that of fluid entering it.
This statement is true. The oncotic pressure in the glomerular capillaries is higher than in the Bowman's capsule, which causes fluid to be filtered out of the capillaries.

Step 2: Evaluate Option B: Glucose concentration in the capillaries is the same as that in glomerular filtrate.
This statement is not true. Glucose is actively reabsorbed in the proximal tubule, so the concentration of glucose in the capillaries is lower than in the glomerular filtrate.

Step 3: Evaluate Option C: Constriction of afferent arteriole decreases the blood flow to the glomerulus.
This statement is true. Constriction of the afferent arteriole reduces the blood flow entering the glomerulus, decreasing the glomerular filtration rate.

Step 4: Evaluate Option D: Hematocrit of the fluid leaving the capillaries is less than that of the fluid entering it.
This statement is true. The glomerular capillaries filter out smaller molecules and fluid, while leaving behind larger molecules and formed elements, such as red blood cells. This results in a lower hematocrit in the fluid leaving the capillaries compared to the entering fluid.

Therefore, the answer is Option B: Glucose concentration in the capillaries is the same as that in glomerular filtrate.
"""
cot_gt = """Ans-a. The oncotic pressure of the fluid leaving the capillaries is less than that of fluid entering it Guyton I LpJ1 4-.;anong 23/e p653-6_)Glomerular oncotic pressure (due to plasma protein content) is higher than that of filtrate oncotic pressure in Bowman's capsule"Since glucose is freely filtered and the fluid in the Bowman's capsule is isotonic with plasma, the concentration of glucose in the filtrate is the same as in the capillaries"""
sentences_to_compare = [
    generated_cot,
    cot_gt
]
sentence_embeddings = model.encode(sentences_to_compare)


In [121]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_similarity(sentence_embeddings[[0]], sentence_embeddings[[1]])

array([[0.82377636]], dtype=float32)

**GPT-4 comparison**
Prompt
```
How similar are the following sentences? Rate the similarity using a score between 0 and 1 with a reasoning for the same.
Sentence A: Step 1: Evaluate Option A: The oncotic pressure of the fluid leaving the capillaries is less than that of fluid entering it.This statement is true. The oncotic pressure in the glomerular capillaries is higher than in the Bowman's capsule, which causes fluid to be filtered out of the capillaries.Step 2: Evaluate Option B: Glucose concentration in the capillaries is the same as that in glomerular filtrate.This statement is not true. Glucose is actively reabsorbed in the proximal tubule, so the concentration of glucose in the capillaries is lower than in the glomerular filtrate.Step 3: Evaluate Option C: Constriction of afferent arteriole decreases the blood flow to the glomerulus.This statement is true. Constriction of the afferent arteriole reduces the blood flow entering the glomerulus, decreasing the glomerular filtration rate.Step 4: Evaluate Option D: Hematocrit of the fluid leaving the capillaries is less than that of the fluid entering it.This statement is true. The glomerular capillaries filter out smaller molecules and fluid, while leaving behind larger molecules and formed elements, such as red blood cells. This results in a lower hematocrit in the fluid leaving the capillaries compared to the entering fluid.Therefore, the answer is Option B: Glucose concentration in the capillaries is the same as that in glomerular filtrate.
Sentence B: Ans-a. The oncotic pressure of the fluid leaving the capillaries is less than that of fluid entering it Guyton I LpJ1 4-.;anong 23/e p653-6_)Glomerular oncotic pressure (due to plasma protein content) is higher than that of filtrate oncotic pressure in Bowman's capsule"Since glucose is freely filtered and the fluid in the Bowman's capsule is isotonic with plasma, the concentration of glucose in the filtrate is the same as in the capillaries
```
Response:
```
The similarity between Sentence A and Sentence B is moderate to high, and I would rate it around 0.7 to 0.8. Both sentences discuss various physiological factors related to the glomerular filtration process in the kidneys, and there is significant overlap in concepts being explained. However, there is a crucial contradiction between the two sentences regarding glucose concentration.

Similar aspects:
1. Both sentences agree that the oncotic pressure of the fluid leaving the capillaries is less than that of fluid entering them, and that the oncotic pressure in the glomerular capillaries is higher than in Bowman's capsule.

2. Both sentences discuss the process of filtration in the kidneys, specifically mentioning how the properties of fluid change as it moves from the capillaries into Bowman's capsule.

Differences:
1. Sentence A incorrectly concludes that glucose concentration in the capillaries is lower than in the glomerular filtrate due to active reabsorption in the proximal tubule. In contrast, Sentence B correctly states that glucose is freely filtered and that the concentration is the same in the filtrate as in the capillaries due to isotonicity.

2. Sentence A provides a broader evaluation, including additional options (C and D) regarding afferent arteriole constriction and hematocrit changes, which are not present in Sentence B.

The contradiction in Sentence A regarding glucose concentration is likely an error, as the explanation within the sentence itself (that glucose is actively reabsorbed in the proximal tubule) does not logically support the conclusion that the concentration in the capillaries would be lower than in the glomerular filtrate initially. The rest of Sentence A aligns well with the content of Sentence B, hence the high similarity score. If not for the error in the conclusion of Sentence A, the similarity could be rated even higher.
```

**Graph Edit distance**

In [122]:
# Evaluating CoT
# candidate_cot_terms = [w.upper() for w in generated_cot.split(" ")]
candidate_cot_terms = ["glucose", "concentration", "capillaries", "glomerular", "filtrate", "oncotic pressure", "afferent arteriole", "blood flow", "glomerulus", "hematocrit", "fluid", "Bowman's capsule", "proximal tubule", "red blood cells"]
cot_terms = []
for candidate in candidate_cot_terms:
    if candidate.upper() in concepts:
        cot_terms.append(candidate.upper())
print(cot_terms)

['GLUCOSE', 'ONCOTIC PRESSURE', 'BLOOD FLOW']


In [123]:
# create graph chaining these terms in the order in which they occur.
cot_graph = nx.DiGraph()
cot_graph.add_nodes_from(cot_terms)
for c in cot_terms:
    if cot_terms.index(c) < len(cot_terms) - 1:
        cot_graph.add_edge(c, cot_terms[cot_terms.index(c) + 1])
print(list(cot_graph.nodes()))
print(list(cot_graph.edges()))

['GLUCOSE', 'ONCOTIC PRESSURE', 'BLOOD FLOW']
[('GLUCOSE', 'ONCOTIC PRESSURE'), ('ONCOTIC PRESSURE', 'BLOOD FLOW')]


In [124]:
# graph edit distance
gedistances = []
for path in paths:
    path_graph = nx.DiGraph()
    path_graph.add_nodes_from(path)
    for c in path:
        if path.index(c) < len(path) - 1:
            path_graph.add_edge(c, path[path.index(c) + 1])
    gedistances.append(nx.graph_edit_distance(path_graph, cot_graph))
print(gedistances)

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [125]:
print(max(gedistances))

0.0


**Comparison of CoT embedding and paths from KG**

Combine the triples from KG paths and compare the cosine similarity of resulting ClinicalBERT embedding against generated CoT embedding.

In [126]:
from transformers import AutoTokenizer, AutoModel
clinicalBert_tokenizer = AutoTokenizer.from_pretrained("medicalai/ClinicalBERT")
clinicalBert_model = AutoModel.from_pretrained("medicalai/ClinicalBERT")

In [127]:
clinicalbert_path_embeddings = []
for path in paths:
    # print(path)
    inputs = clinicalBert_tokenizer(" ".join(path), return_tensors="pt")
    outputs = clinicalBert_model(**inputs)
    clinicalbert_path_embeddings.append(outputs.last_hidden_state.mean(dim=1))
print(torch.cat(clinicalbert_path_embeddings, dim=0).shape)
print(len(paths))


torch.Size([82, 768])
82


In [128]:
generated_cot_clinicalbert_embedding = clinicalBert_model(**clinicalBert_tokenizer(generated_cot, return_tensors="pt")).last_hidden_state.mean(dim=1)
generated_cot_clinicalbert_embedding.shape

torch.Size([1, 768])

In [129]:
similarity_cot_graph_paths = []
for i in clinicalbert_path_embeddings:
    cosine_sim = (torch.nn.functional.cosine_similarity(generated_cot_clinicalbert_embedding, i))
    similarity_cot_graph_paths.append(cosine_sim.item())
print(max(similarity_cot_graph_paths))

0.6146697402000427


**BLEU/Rouge**

In [130]:
bleu = bleu_score.compute(predictions=[generated_cot], references=[cot_gt])
rouge = rouge_score.compute(predictions=[generated_cot], references=[cot_gt])
print(bleu)
print(rouge)

{'bleu': 0.13507942108928866, 'precisions': [0.22448979591836735, 0.1557377049180328, 0.11522633744855967, 0.08264462809917356], 'brevity_penalty': 1.0, 'length_ratio': 2.784090909090909, 'translation_length': 245, 'reference_length': 88}
{'rouge1': 0.37837837837837834, 'rouge2': 0.26530612244897955, 'rougeL': 0.30405405405405406, 'rougeLsum': 0.3108108108108108}


### 1-shot CoT with answer in a template form
Prompt:
```
1. <step1>
2. <step2>
...
So the answer is (<answer>).
Make sure that the answer uses the above format. Answer the question step by step.

Question: Low insulin to glucagon ratio is seen in all of these except:
A) Glycogen synthesis
B) Glycogen breakdown
C) Gluconeogenesis
D) Ketogenesis
1. Low insulin to glucagon ratio implies a catabolic state, i.e. usage of body stores to form energy, typically seen in fasting state ancl diabetes mellitus.
2. Catabolic state promotes glycogen breakdown, gluconeogenesis and ketogenesis.
3. Glycogen synthesis is an anabolic process, which is promoted by insulin.
So the answer is A.

Question: Which of the following is not true about glomerular capillaries?
A) The oncotic pressure of the fluid leaving the capillaries is less than that of fluid entering it
B) Glucose concentration in the capillaries is the same as that in glomerular filtrate
C) Constriction of afferent aeriole decreases the blood flow to the glomerulas
D) Hematocrit of the fluid leaving the capillaries is less than that of the fluid entering it
```
Answer:
```
1. Evaluate Option A: The oncotic pressure of the fluid leaving the capillaries is less than that of fluid entering it.
This statement is true. The oncotic pressure in the glomerular capillaries is higher than in the Bowman's capsule, causing fluid to be filtered out of the capillaries.

2. Evaluate Option B: Glucose concentration in the capillaries is the same as that in glomerular filtrate.
This statement is not true. Glucose is actively reabsorbed in the proximal tubule, so the concentration of glucose in the capillaries is lower than in the glomerular filtrate.

3. Evaluate Option C: Constriction of afferent arteriole decreases the blood flow to the glomerulus.
This statement is true. Constriction of the afferent arteriole reduces the blood flow entering the glomerulus, decreasing the glomerular filtration rate.

4. Evaluate Option D: Hematocrit of the fluid leaving the capillaries is less than that of the fluid entering it.
This statement is true. The glomerular capillaries filter out smaller molecules and fluid, while leaving behind larger molecules and formed elements, such as red blood cells. This results in a lower hematocrit in the fluid leaving the capillaries compared to the entering fluid.

Therefore, the answer is Option B: Glucose concentration in the capillaries is the same as that in glomerular filtrate.
```

**Sentence BERT comparison**

In [131]:
generated_cot = """
1. Evaluate Option A: The oncotic pressure of the fluid leaving the capillaries is less than that of fluid entering it.
This statement is true. The oncotic pressure in the glomerular capillaries is higher than in the Bowman's capsule, causing fluid to be filtered out of the capillaries.

2. Evaluate Option B: Glucose concentration in the capillaries is the same as that in glomerular filtrate.
This statement is not true. Glucose is actively reabsorbed in the proximal tubule, so the concentration of glucose in the capillaries is lower than in the glomerular filtrate.

3. Evaluate Option C: Constriction of afferent arteriole decreases the blood flow to the glomerulus.
This statement is true. Constriction of the afferent arteriole reduces the blood flow entering the glomerulus, decreasing the glomerular filtration rate.

4. Evaluate Option D: Hematocrit of the fluid leaving the capillaries is less than that of the fluid entering it.
This statement is true. The glomerular capillaries filter out smaller molecules and fluid, while leaving behind larger molecules and formed elements, such as red blood cells. This results in a lower hematocrit in the fluid leaving the capillaries compared to the entering fluid.

Therefore, the answer is Option B: Glucose concentration in the capillaries is the same as that in glomerular filtrate.
"""
cot_gt = """Ans-a. The oncotic pressure of the fluid leaving the capillaries is less than that of fluid entering it Guyton I LpJ1 4-.;anong 23/e p653-6_)Glomerular oncotic pressure (due to plasma protein content) is higher than that of filtrate oncotic pressure in Bowman's capsule"Since glucose is freely filtered and the fluid in the Bowman's capsule is isotonic with plasma, the concentration of glucose in the filtrate is the same as in the capillaries"""
sentences_to_compare = [
    generated_cot,
    cot_gt
]
sentence_embeddings = model.encode(sentences_to_compare)


In [132]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_similarity(sentence_embeddings[[0]], sentence_embeddings[[1]])

array([[0.79697204]], dtype=float32)

**GPT-4 comparison**
Prompt
```
How similar are the following sentences? Rate the similarity using a score between 0 and 1 with a reasoning for the same.
Sentence A: 1. Evaluate Option A: The oncotic pressure of the fluid leaving the capillaries is less than that of fluid entering it.This statement is true. The oncotic pressure in the glomerular capillaries is higher than in the Bowman's capsule, causing fluid to be filtered out of the capillaries.2. Evaluate Option B: Glucose concentration in the capillaries is the same as that in glomerular filtrate.This statement is not true. Glucose is actively reabsorbed in the proximal tubule, so the concentration of glucose in the capillaries is lower than in the glomerular filtrate.3. Evaluate Option C: Constriction of afferent arteriole decreases the blood flow to the glomerulus.This statement is true. Constriction of the afferent arteriole reduces the blood flow entering the glomerulus, decreasing the glomerular filtration rate.4. Evaluate Option D: Hematocrit of the fluid leaving the capillaries is less than that of the fluid entering it.This statement is true. The glomerular capillaries filter out smaller molecules and fluid, while leaving behind larger molecules and formed elements, such as red blood cells. This results in a lower hematocrit in the fluid leaving the capillaries compared to the entering fluid.Therefore, the answer is Option B: Glucose concentration in the capillaries is the same as that in glomerular filtrate.
Sentence B: Ans-a. The oncotic pressure of the fluid leaving the capillaries is less than that of fluid entering it Guyton I LpJ1 4-.;anong 23/e p653-6_)Glomerular oncotic pressure (due to plasma protein content) is higher than that of filtrate oncotic pressure in Bowman's capsule"Since glucose is freely filtered and the fluid in the Bowman's capsule is isotonic with plasma, the concentration of glucose in the filtrate is the same as in the capillaries
```
Response
```
The similarity between Sentence A and Sentence B is moderate and I would rate it around 0.5 to 0.6. Here is the reasoning for the rating:

Both sentences discuss the physiology of the glomerular filtration process, specifically mentioning oncotic pressure and glucose concentration. They contain several key points of overlap but also have a critical discrepancy:

Similarities:
- Both sentences state that the oncotic pressure in the glomerular capillaries is higher than that in Bowman's capsule, leading to fluid being filtered out of the capillaries.
- The sentences address the concept of glucose concentration in relation to the capillaries and the glomerular filtrate.

Differences:
- Sentence A contains an incorrect statement regarding glucose concentration, claiming that the concentration in the capillaries is lower than in the glomerular filtrate due to reabsorption in the proximal tubule. This is a misunderstanding, as reabsorption occurs after the filtrate has been formed, and initially, the concentration of glucose is the same in the capillaries and the filtrate.
- Sentence B correctly states that glucose concentration is the same in the filtrate and the capillaries, as the filtrate is isotonic with plasma.
- Sentence A erroneously concludes that glucose concentration in the capillaries and glomerular filtrate is the same, which is the opposite of its earlier incorrect statement.
- Sentence A evaluates multiple options related to kidney physiology, while Sentence B focuses solely on oncotic pressure and glucose concentration.

The final scores are tempered by the contradiction in Sentence A, which affects the overall similarity. If Sentence A's conclusion about glucose concentration was consistent with its previous assertion (even though it would still be incorrect), the similarity score would be lower due to the clear discrepancy between the two sentences. The moderate similarity score reflects the mixture of aligned information about oncotic pressure and contrasting statements about glucose concentration.
```

**Graph Edit distance**

In [133]:
# Evaluating CoT
# candidate_cot_terms = [w.upper() for w in generated_cot.split(" ")]
candidate_cot_terms = ["glucose", "concentration", "capillaries", "glomerular", "filtrate", "oncotic pressure", "afferent arteriole", "blood flow", "glomerulus", "hematocrit", "fluid", "Bowman's capsule", "proximal tubule", "red blood cells"]
cot_terms = []
for candidate in candidate_cot_terms:
    if candidate.upper() in concepts:
        cot_terms.append(candidate.upper())
print(cot_terms)

['GLUCOSE', 'ONCOTIC PRESSURE', 'BLOOD FLOW']


In [134]:
# create graph chaining these terms in the order in which they occur.
cot_graph = nx.DiGraph()
cot_graph.add_nodes_from(cot_terms)
for c in cot_terms:
    if cot_terms.index(c) < len(cot_terms) - 1:
        cot_graph.add_edge(c, cot_terms[cot_terms.index(c) + 1])
print(list(cot_graph.nodes()))
print(list(cot_graph.edges()))

['GLUCOSE', 'ONCOTIC PRESSURE', 'BLOOD FLOW']
[('GLUCOSE', 'ONCOTIC PRESSURE'), ('ONCOTIC PRESSURE', 'BLOOD FLOW')]


In [135]:
# graph edit distance
gedistances = []
for path in paths:
    path_graph = nx.DiGraph()
    path_graph.add_nodes_from(path)
    for c in path:
        if path.index(c) < len(path) - 1:
            path_graph.add_edge(c, path[path.index(c) + 1])
    gedistances.append(nx.graph_edit_distance(path_graph, cot_graph))
print(gedistances)

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [136]:
print(max(gedistances))

0.0


**Comparison of CoT embedding and paths from KG**

Combine the triples from KG paths and compare the cosine similarity of resulting ClinicalBERT embedding against generated CoT embedding.

In [137]:
from transformers import AutoTokenizer, AutoModel
clinicalBert_tokenizer = AutoTokenizer.from_pretrained("medicalai/ClinicalBERT")
clinicalBert_model = AutoModel.from_pretrained("medicalai/ClinicalBERT")

In [138]:
clinicalbert_path_embeddings = []
for path in paths:
    # print(path)
    inputs = clinicalBert_tokenizer(" ".join(path), return_tensors="pt")
    outputs = clinicalBert_model(**inputs)
    clinicalbert_path_embeddings.append(outputs.last_hidden_state.mean(dim=1))
print(torch.cat(clinicalbert_path_embeddings, dim=0).shape)
print(len(paths))


torch.Size([82, 768])
82


In [139]:
generated_cot_clinicalbert_embedding = clinicalBert_model(**clinicalBert_tokenizer(generated_cot, return_tensors="pt")).last_hidden_state.mean(dim=1)
generated_cot_clinicalbert_embedding.shape

torch.Size([1, 768])

In [140]:
similarity_cot_graph_paths = []
for i in clinicalbert_path_embeddings:
    cosine_sim = (torch.nn.functional.cosine_similarity(generated_cot_clinicalbert_embedding, i))
    similarity_cot_graph_paths.append(cosine_sim.item())
print(max(similarity_cot_graph_paths))

0.6121519207954407


**BLEU/Rouge**

In [141]:
bleu = bleu_score.compute(predictions=[generated_cot], references=[cot_gt])
rouge = rouge_score.compute(predictions=[generated_cot], references=[cot_gt])
print(bleu)
print(rouge)

{'bleu': 0.1379113363747685, 'precisions': [0.22916666666666666, 0.1589958158995816, 0.11764705882352941, 0.08438818565400844], 'brevity_penalty': 1.0, 'length_ratio': 2.727272727272727, 'translation_length': 240, 'reference_length': 88}
{'rouge1': 0.3848797250859107, 'rouge2': 0.26989619377162627, 'rougeL': 0.30927835051546393, 'rougeLsum': 0.3161512027491408}
